In [1]:
%cd ../../
%pwd

/home/jupyter-vincent/6-bap


/opt/tljh/user/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/jupyter-vincent/6-bap'

In [2]:
from statistics import mean
from chatbot_domain.benchmark import JsonQuestionParser, Benchmarker, AlwaysATestSubject, NLPTestSubject, HumanTestSubject, TestScore
from chatbot_domain.chatbot import ChatBotModifier, OpenAIChatBot, DIPDomainGuard, ChatBotBuilder, RAGBuilder
from chatbot_domain.rag import VectorRetriever, FacebookDPR
from chatbot_domain.transformers import ModelBuilder
from chatbot_domain.data import loadDataSetFromDisk
from chatbot_domain import logger
import os
import datetime
import torch
import json

logger.setLevel("INFO")
parser = JsonQuestionParser('questions.json')
questions = parser.parse()

def runTest(scoreDirectory: str, testRunName: str, benchmarker: Benchmarker, summary : dict,samples: int = 3) -> list[TestScore]:
    scores : list[TestScore] =  []
    logger.info(f"Starting tests for {testRunName}")
    summarySection = {
        'scores': []
    }
    
    for i in range(samples):
        logger.info(f"Run {i+1}/{samples}")
        result = benchmarker.evaluate()
        scores.append(result)
        result.save(f"{scoreDirectory}/{testRunName}")
        summarySection['sores'].append(result.total)
        print(f"{testRunName} run {i+1}:\n{result}")
    average = mean([score.average for score in scores])
    summarySection['average'] = average
    summary[testRunName] = summarySection
    print(f"Average score for {testRunName}: {average}")
    return scores
            
globalScoreDirectory = "./scores"            
testRunDirectory = None
i = 0
while testRunDirectory is None:
    proposal = f"{globalScoreDirectory}/{datetime.date.today().strftime('%d-%m-%Y')}-{i}"
    if not os.path.exists(proposal):
        testRunDirectory = proposal
        break
    i += 1
scores = []
summary = {}
# OpenAI models with RAG
# chatbot = ChatBotBuilder.openAI('gpt-4-turbo').benchmarkGuard().domainGuard(DIPDomainGuard).rag(
#     RAGBuilder.fromDatasetDisk('ragData.set').vectorRetriever(FacebookDPR()), contextLength = 1024 * 8
#     ).build()
# benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)

# None Finetuned Model without RAG
chatbot = ChatBotBuilder.model(
    ModelBuilder().modelName('mistralai/Mixtral-8x7B-Instruct-v0.1').deviceMap("auto").shouldQuantize(True).build(adapted=False)
    ).benchmarkGuard().domainGuard(DIPDomainGuard).build()
benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)
scores.extend(runTest(testRunDirectory, "mixtral-8*7B", benchmarker, summary))
del chatbot
torch.cuda.empty_cache()

# None Finetuned Model with RAG
chatbot = ChatBotBuilder.model(
    ModelBuilder().modelName('mistralai/Mixtral-8x7B-Instruct-v0.1').deviceMap("auto").shouldQuantize(True).build(adapted=False)
    ).benchmarkGuard().domainGuard(DIPDomainGuard).rag(
        RAGBuilder.fromDatasetDisk('ragData.set').vectorRetriever(FacebookDPR()), contextLength= 2048
    ).build()
benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)
scores.extend(runTest(testRunDirectory, "mixtral-8*7B-RAG2k", benchmarker, summary))
del chatbot
torch.cuda.empty_cache()


# None Finetuned Model without RAG
chatbot = ChatBotBuilder.model(
    ModelBuilder().modelName('mistralai/Mistral-7B-Instruct-v0.2').deviceMap("auto").shouldQuantize(True).build(adapted=False)
    ).benchmarkGuard().domainGuard(DIPDomainGuard).build()
benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)
testName = "mistral7B"
scores.extend(runTest(testRunDirectory, "mistral7B", benchmarker, summary))
del chatbot
torch.cuda.empty_cache()

# None Finetuned Model with RAG
chatbot = ChatBotBuilder.model(
    ModelBuilder().modelName('mistralai/Mistral-7B-Instruct-v0.2').deviceMap("auto").shouldQuantize(True).build(adapted=False)
    ).benchmarkGuard().domainGuard(DIPDomainGuard).rag(
        RAGBuilder.fromDatasetDisk('ragData.set').vectorRetriever(FacebookDPR()), contextLength= 2048
    ).build()
benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)
scores.extend(runTest(testRunDirectory, "mistral7B-RAG2k", benchmarker, summary))
del chatbot
torch.cuda.empty_cache()

# Finetuned Model without RAG
chatbot = ChatBotBuilder.model(
    ModelBuilder().modelName('vincentverbergt/Mistral7B-DIP').deviceMap("auto").shouldQuantize(True).build(adapted=True)
    ).benchmarkGuard().domainGuard(DIPDomainGuard).build()
benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)
scores.extend(runTest(testRunDirectory, "mistral7B-ft", benchmarker, summary))
del chatbot
torch.cuda.empty_cache()

# Finetuned Model with RAG
chatbot = ChatBotBuilder.model(
    ModelBuilder().modelName('vincentverbergt/Mistral7B-DIP').deviceMap("auto").shouldQuantize(True).build(adapted=True)
    ).benchmarkGuard().domainGuard(DIPDomainGuard).rag(
        RAGBuilder.fromDatasetDisk('ragData.set').vectorRetriever(FacebookDPR()), contextLength= 2048
    ).build()
benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)
scores.extend(runTest(testRunDirectory, "mistral7B-ft-RAG2k", benchmarker, summary))
del chatbot
torch.cuda.empty_cache()


# Human Test subject
# name = input("name: ")
# benchmarker = Benchmarker(HumanTestSubject(name), questions)


print(scores)
print(summary)
with open(f"{testRunDirectory}/summary.json", 'w') as file:
    jsonText = json.dumps(scores)
    file.write(jsonText)

2024-04-23 12:23:46,156 - chatbot_domain - INFO - loading model
2024-04-23 12:23:46,156 - chatbot_domain - INFO - With Quantization


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.
2024-04-23 12:23:51,162 - chatbot_domain - INFO - Loading tokenizer
2024-04-23 12:23:51,294 - chatbot_domain - INFO - Loading dataset from ragData.set
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (ini

  0%|          | 0/16 [00:00<?, ?it/s]

2024-04-23 12:23:55,103 - chatbot_domain - INFO - Saving modified dataset
2024-04-23 12:23:55,104 - chatbot_domain - INFO - Question 1/23
2024-04-23 12:23:55,104 - chatbot_domain - DEBUG - The final prompt is If radiometric quantities talk about how we perceive light, will it cover more or less of the spectrum with respect to photometric quantities?
1. Yes.
2. No.
3. No correct answer is present.
4. This question is outside my domain of knowledge.

2024-04-23 12:23:55,268 - chatbot_domain - DEBUG - Context concatenation loop completed without breaking, updating sampleGuess to 20
2024-04-23 12:23:55,269 - chatbot_domain - DEBUG - rerunning context gathering with higher samples
2024-04-23 12:23:55,284 - chatbot_domain - DEBUG - Context concatenation loop completed without breaking, updating sampleGuess to 40
2024-04-23 12:23:55,284 - chatbot_domain - DEBUG - rerunning context gathering with higher samples
2024-04-23 12:23:55,303 - chatbot_domain - DEBUG - Context concatenation loop compl


    # TEST RESULTS
    overal Score: 8/23
    ## breakdown:
        literal:                 6/13
        deduction:               1/3
        wrong answers:           0/4
        outside domain:          1/3        
    
